<a href="https://www.kaggle.com/code/basth94/ml-bio-exercice-12?scriptVersionId=144223851" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Exercice 12:

## Création d'un CNN pour la classification des scaleogrammes

[lien Nowledgeable](https://nowledgeable.com/invitation/student/join-module/9c8ec467-686a-44cd-a2f2-85cf174a79ad) <br>
A l'aide de la transformée en ondelettes continue, transformez des signaux ECG en scaleograms et puis utilisez un modèle CNN pour classifier des images.


In [ ]:
!pip install wfdb

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
pd.set_option('display.max_columns', None)

import wfdb
import glob
import os
import pywt
import tensorflow as tf
from tqdm import tqdm
import tensorflow as tf
from scipy import signal as scipy_signal
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import  Dense, Dropout, Conv2D, Input, MaxPooling2D,Flatten


Afin de pouvoir récupérer les signaux issus des datasets de physionet nous réutiliserons la fonction `load_signal_using_wfdb`

In [ ]:
def load_signal_using_wfdb(file, start, end, channel, pn_dir):
    """
    file: name of record
    start: int
    end:int
    channel: 0 or 1
    """
    record = wfdb.rdrecord(file, sampfrom = start, sampto = end, channels=[channel], pn_dir=pn_dir)
    data = record.p_signal.reshape(-1)
    return data


Nous allons étudier 3 Datasets:
- `nsrdb` : Normal Sinus Rhythm Database, les rythmes cardiaques normaux.
- `mitdb` : Les rythmes cardiaques pour les personnes faisant de l'arythmie.
- `chfdb` : Les rythmes cardiaques pour les personnes ayant des défaillances cardiaques.

Le but va être de générer un modèle convolutif comme suit:
- Récupération des records.
- Récupération des fréquences.
- Labelisation des données.
- Extraction et synchronisation des signaux issus de chaque record.
- Découpage des signaux.
- Récupération de leur CWT.
- Passage dans le modèle convolutif.

In [ ]:
database_names = ['nsrdb','mitdb','chfdb']
n_label = len(database_names)

In [ ]:
record_names = []
labels = []

for i,name in enumerate(database_names):
    current_record_names = wfdb.get_record_list(name)
    record_names += current_record_names
    labels += [i for record_name in current_record_names]
assert len(record_names) == len(labels)

Nous récupèrons les fréquences pour permettre une bonne synchronisation.

In [ ]:
frequencies = []
for record_name,label in tqdm(zip(record_names,labels)):
    header = wfdb.rdheader(record_name,database_names[label])
    frequencies.append(header.fs)


In [ ]:
def synchronize_signal(signal, sampling_rate, common_sampling_rate):
    

    num_samples = int((common_sampling_rate*len(signal))/sampling_rate)
    signal_resampled = scipy_signal.resample(signal, num_samples)

    return signal_resampled

Passons à l'extraction de la donnée:

In [ ]:
def generate_data(sample_size,scale_size,nb_samples_per_signal , labels, record_names,database_names, file_path = None, frequencies = [] ,common_sampling_rate = 128 ):
    
    scales = range(1, scale_size+1)
    waveletname = 'morl'
    signal_ds = []
    
    y_train = np.zeros(nb_samples_per_signal*len(labels))
    signal_ds = []
    
    for i in tqdm(range(len(record_names))):
        record_name,label,frequency = record_names[i],labels[i],frequencies[i]
        pn_dir = database_names[label]
        signal=(load_signal_using_wfdb(record_name, start=0, end=nb_samples_per_signal*frequency, channel = 0, pn_dir=pn_dir))
        signal = synchronize_signal(signal,frequency,common_sampling_rate)
        signal_ds += np.split(signal,nb_samples_per_signal)
        y_train[i*nb_samples_per_signal:(i+1)*nb_samples_per_signal] = label

    signal_ds = np.array(signal_ds)
    
    X_train = []
    for signal in signal_ds:
        coeff, freqs = pywt.cwt(signal, scales, waveletname, 1)
        X_train.append(coeff)
    X_train = np.array(X_train)
    X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],X_train.shape[2],1))
    X_tensor = tf.convert_to_tensor(X_train)

    num_classes = len(np.unique(y_train))
    y = tf.keras.utils.to_categorical(y_train,num_classes)
    
    y_tensor = tf.convert_to_tensor(y)
    print(y_tensor[0])
    return X_tensor,y_tensor


Générons maintenant les données `X_tensor` et leurs label `y_tensor`

In [ ]:

sample_size = 64
scale_size = 64
nb_samples_per_signal = 64
common_sampling_rate = 64

X_tensor, y_tensor = generate_data(sample_size = sample_size,scale_size = scale_size,nb_samples_per_signal = nb_samples_per_signal, labels= labels, record_names = record_names,database_names=database_names,file_path='signals_csv',frequencies = frequencies ,common_sampling_rate = common_sampling_rate)
assert X_tensor.shape[0] == y_tensor.shape[0]

In [ ]:
def build_model(input_shape,num_class):
    model = Sequential()
    model.add(Input(shape=input_shape))
    
    model.add(Conv2D(filters=64,kernel_size=5, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters=32,kernel_size=3, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Dropout(0.5))
    
    model.add(Conv2D(filters=32,kernel_size=3, activation='relu'))
    model.add(MaxPooling2D())

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_class,activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model
model = build_model(X_tensor.shape[1:],y_tensor.shape[1])
model.summary()

In [ ]:
X_tensor.shape,y_tensor.shape

In [ ]:
model.fit(X_tensor,y_tensor,validation_split=0.2,epochs=5)

Nous pouvons hélas remarquer que la validation accuracy ne s'élve pas là où l'accuracy de training converge vers 1, le modèle overfit il n'est soit pas adapté aux données soit les données ont été mal générées en terme de choix de dimensions où d'échantillonage.